In [1]:
# Imports
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset

In [2]:
# Create Fully Connected Network
class NN(nn.Module):
    def __init__(self,input_size,num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size ,50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')


In [4]:
# Hyperparameter
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1


In [5]:
!pwd

/kaggle/working


In [6]:
!mkdir dataset
!ls

__notebook_source__.ipynb  dataset


In [7]:
# Load Data
train_dataset = datasets.MNIST(root = '/kaggle/working/dataset/',train = True,transform = transforms.ToTensor(),download = True)


Extracting /kaggle/working/dataset/MNIST/raw/train-images-idx3-ubyte.gz to /kaggle/working/dataset/MNIST/raw


Extracting /kaggle/working/dataset/MNIST/raw/train-labels-idx1-ubyte.gz to /kaggle/working/dataset/MNIST/raw


Extracting /kaggle/working/dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to /kaggle/working/dataset/MNIST/raw




Extracting /kaggle/working/dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to /kaggle/working/dataset/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [8]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: /kaggle/working/dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [9]:
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle = True)
test_dataset = datasets.MNIST(root ='/kaggle/working/dataset/',train = False,transform = transforms.ToTensor(),download = True)
test_loader = DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle = True)

In [11]:
#Initialize network
model = NN(input_size = input_size,num_classes = num_classes).to(device)

In [12]:
#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)


In [15]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        # get data to cuda if possible
        data = data.to(device = device)
        targets = targets.to(device = device)
        
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        


In [16]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            x = x.reshape(x.shape[0],-1)
            
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        
    model.train()
    
check_accuracy(train_loader, model)
check_accuracy(test_loader,model)


NameError: name 'loader' is not defined